## AI Planning For Robot Systems Assignment 2

<p>In this Notebook there is an implementation of a scenario of a simple world that contains some objects inside an inner room and the robot supposed to move these objects into their distenation area. </p>

### We Start by importing the libraries and defining the global variables

In this Program the (tkinter) Python Library is used to build the GUI.

In [1]:
from tkinter import *
from time import sleep


win = Tk()
canvas = Canvas(width=500, height=500)
gridsize = 50
start = ()
box = ()
goal = ()
startList = []
boxList = []
goalList = []

### Then we define the Robot Class which contains:
<ul>
    <li><strong>uploadWorld():</strong> to make the robot visualize the world</li>
    <li><strong>Move_to():</strong> which is the search algorithm to make the robot find the shortest path to the distenation assigned using the Breadth First Search Algorithm</li>
    <li><strong>pick_up():</strong> giving the pickup order to the robot and updating the robot state</li>
    <li><strong>putdown():</strong> giving the putdown order to the robot and updating the robot state</li>
</ul>

**Then there is the trigger function called apply_task which maked the robot starts operating**


In [2]:
class Robot:
    def __init__(self):
        self.grid = {}
        self.armEmpty = True
        self.holding = False       
    
    def uploadWorld(self, key, value):
        if self.grid.get(key) != None:
            gridEdges = self.grid[key]
            gridEdges.append(value)
        else:
            gridEdges = []
            gridEdges.append(value)
        self.grid[key] = gridEdges

    def Move_to(self, start, box, visited, stack):
        queue = [[start]]
        global paths
        visited = set()

        while queue:
            path = queue.pop(0)
            cell = path[-1]
            if cell == box:
                path = path[1:-1]
                for cell in path:
                    win.update()
                    canvas.create_rectangle(cell[0]-25, cell[1]-25, cell[0]+25, cell[1]+25, fill="skyblue")
                    sleep(0.1)
                sleep(0.1)
                for cell in path:
                    win.update()
                    canvas.create_rectangle(cell[0]-25, cell[1]-25, cell[0]+25, cell[1]+25, fill="white")
                    sleep(0.1)
                return
            elif cell not in visited:
                for adjacent in self.grid[cell][0]:
                    new_path = list(path)
                    new_path.append(adjacent)
                    queue.append(new_path)
                visited.add(cell)

    def pick_up(self, cell):
        self.armEmpty = False
        self.holding = True
        print("Picking the Object")
        canvas.create_rectangle(cell[0]-25, cell[1]-25, cell[0]+25, cell[1]+25, fill="white")
        print("Status Update: Holding Object\n")
        
        
    def putdown(self, cell):
        self.armEmpty = True
        self.holding = False
        print("Putting down the Object")
        canvas.create_rectangle(cell[0]-25, cell[1]-25, cell[0]+25, cell[1]+25, fill="#F01919")
        print("Status Update: Arms Empty\n")
        
robot = Robot()

def apply_task():
    global start
    global box
    global goal
    visited = set()
    start = start[0: -1]

    for i in range(0,len(goal)):
        robot.Move_to(start[i], box[i], visited, [])
        robot.pick_up(box[i])
        robot.Move_to(box[i], goal[i], visited, [])
        robot.putdown(goal[i])
        canvas.create_rectangle(start[i][0]-25, start[i][1]-25, start[i][1]+25, start[i][1]+25)
    start = ()
    box = ()
    goal = ()

### Here After we defined the Robot Class and its function we create the world in a grid representation
<ul>
    <li><strong>surroundingCells():</strong> Updates the robot of which cells surrounding it at the moment which it can move through.</li>
    <li><strong>PlotCriticalPoint():</strong> This Function responds to the event done by the user in order to define the start point, object locations, and drop areas.</li>
    <li><strong>Create_obstacle():</strong> This Function responds to the event done by the user in order to make a certain cell in the grid acts as an obstacle so the robot can't move throught it.</li>
</ul>


In [3]:
def surroundingCells(edges):
    surroundingCells = []
    for edge in edges:
        if edge[0] > 0 and edge[0] < 500 and edge[1] > 0 and edge[1] < 500:
            surroundingCells.append(edge)
    return surroundingCells

# Drawing the world in a Grid Representation and Telling the Robot Which Cells are Valid to move through
x,y = 0,0
for i in range(36):
    for j in range(36):
        canvas.create_rectangle(x, y, x+gridsize, y+gridsize,  fill="white")
        edges = []

        midpointx = x + gridsize / 2
        midpointy = y + gridsize / 2

        key = (midpointx, midpointy)

        # ADJACENT CELLS
        TopCell = (midpointx, midpointy - 50)
        BottomCell = (midpointx, midpointy + 50)
        RightCell = (midpointx + 50, midpointy)
        LeftCell = (midpointx - 50, midpointy)


        edges = surroundingCells([TopCell,BottomCell,RightCell,LeftCell])

        robot.uploadWorld(key=key, value=edges)
        x = x + gridsize
    x = 0
    y = y + gridsize

def plotCriticalPoints(event,pick):
    global start
    global box
    global goal

    x2 = event.x
    while x2%gridsize != 0:
        x2 += 1
    x1 = x2 - gridsize

    y2 = event.y
    while y2%gridsize != 0:
        y2 += 1
    y1 = y2 - gridsize

    if pick == 'start':
        canvas.create_rectangle(x1,y1,x2,y2,fill='#17F63C')
        s = (x1+gridsize/2, y1+gridsize/2)
        startList = list(start)
        startList.append(s)
        start = tuple(startList)

    elif pick == 'box':
        canvas.create_rectangle(x1,y1,x2,y2,fill='#F01919')
        b = (x1+gridsize/2, y1+gridsize/2)
        boxList = list(box)
        boxList.append(b)
        box = tuple(boxList)
    elif pick == 'goal':
        canvas.create_rectangle(x1,y1,x2,y2,fill='#8A2BE2')
        g = (x1+gridsize/2, y1+gridsize/2)
        goalList = list(goal)
        startList = list(start)
        goalList.append(g)
        startList.append(g)
        goal = tuple(goalList)
        start = tuple(startList)

def create_obstacle(event):
    x2 = event.x
    while x2%gridsize != 0:
        x2 += 1
    x1 = x2 - gridsize

    y2 = event.y
    while y2%gridsize != 0:
        y2 += 1
    y1 = y2 - gridsize
    canvas.create_rectangle(x1,y1,x2,y2,fill='#333333')

    point = (x2-25, y2-25)
    if point in robot.grid.keys():
        deleteflist = robot.grid[point][0]
        robot.grid.pop(point)
        for el in deleteflist:
            robot.grid[el][0].remove(point)
            
def popupmsg(msg):
    popup = Tk()
    
    def leavemini():
        popup.destroy()
    
    popup.wm_title("Program Instructions")
    label = Label(popup, text = msg)
    label.pack(side = "top", fill = "x", pady = 10)
    B1 = Button(popup, text= "Confirm", command = leavemini, fg='white', bg='#5852ad')
    B1.pack()

### The Main:
**Contains the Mouse events and the popup message are specifed, the GUI is Built, and the Program is Initiated.**

In [4]:
ppmsg = "Below are the Instructions to use the program. \n1: Press (Ctrl + Left Mouse Click) to Specify the Starting Point (Press once). \n2: Press (Ctrl + Right Mouse Click) to Specify the Locations of the Objects. \n3: Press (Alt + Left Mouse Click) to Specify the Drop Locations. \n4: (Drag and Press Left Mouse click) to Create An Obstacle."
popupmsg(ppmsg)
print(ppmsg)

button = Button(win, text='Start',fg='white', bg='#5852ad',command=apply_task)
button.pack(side=BOTTOM)

win.bind("<B1-Motion>", create_obstacle)
win.bind("<Control-Button-3>", lambda e: plotCriticalPoints(event=e, pick='box'))
win.bind("<Control-Button-1>", lambda e: plotCriticalPoints(event=e, pick='start'))
win.bind("<Alt-Button-1>", lambda e: plotCriticalPoints(event=e, pick='goal'))

canvas.pack()
win.mainloop()

Below are the Instructions to use the program. 
1: Press (Ctrl + Left Mouse Click) to Specify the Starting Point (Press once). 
2: Press (Ctrl + Right Mouse Click) to Specify the Locations of the Objects. 
3: Press (Alt + Left Mouse Click) to Specify the Drop Locations. 
4: (Drag and Press Left Mouse click) to Create An Obstacle.
Picking the Object
Status Update: Holding Object

Putting down the Object
Status Update: Arms Empty

Picking the Object
Status Update: Holding Object

Putting down the Object
Status Update: Arms Empty

Picking the Object
Status Update: Holding Object

Putting down the Object
Status Update: Arms Empty

